# Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

Acquire and Summarize

Acquire data from the cloud database.
You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.


Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.


Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import wrangle

####
#creating a connect function to connected to the code up servers
def get_connection(db, user=env.user, host=env.host, password=env.password):
    '''initiates sql connection'''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
#pulling from the code up server the zillow data frame   
def sqlclean_zillow():
    query = '''
    SELECT
    prop.*,
    predictions_2017.logerror,
    predictions_2017.transactiondate,
    air.airconditioningdesc,
    arch.architecturalstyledesc,
    build.buildingclassdesc,
    heat.heatingorsystemdesc,
    landuse.propertylandusedesc,
    story.storydesc,
    construct.typeconstructiondesc
    FROM properties_2017 prop
    JOIN (
    SELECT parcelid, MAX(transactiondate) AS max_transactiondate
    FROM predictions_2017
    GROUP BY parcelid
    ) pred USING(parcelid)
    JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
    AND pred.max_transactiondate = predictions_2017.transactiondate
    LEFT JOIN airconditioningtype air USING (airconditioningtypeid)
    LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
    LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
    LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid)
    LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
    LEFT JOIN storytype story USING (storytypeid)
    LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid)
    WHERE prop.latitude IS NOT NULL
    AND prop.longitude IS NOT NULL
    AND transactiondate <= '2017-12-31' '''

    url = f"mysql+pymysql://{env.user}:{env.password}@{env.host}/zillow"
    df = pd.read_sql(query,url)

    return df

In [2]:
df=sqlclean_zillow()
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,None,None,Central,Condominium,None,None


In [51]:
def overview(df):
    print('--- Shape: {}'.format(df.shape))
    print('--- Info')
    df.info()
    print('--- Column Descriptions')
    print(df.describe(include='all'))

def nulls_by_columns(df):
    return pd.concat([
        df.isna().sum().rename('count'),
        df.isna().mean().rename('percent')
    ], axis=1)

def nulls_by_rows(df):
    return pd.concat([
        df.isna().sum(axis=1).rename('n_missing'),
        df.isna().mean(axis=1).rename('percent_missing'),
    ], axis=1).value_counts().sort_index()

def handle_missing_values(df, prop_required_column, prop_required_row):
    n_required_column = round(df.shape[0] * prop_required_column)
    n_required_row = round(df.shape[1] * prop_required_row)
    df = df.dropna(axis=0, thresh=n_required_row)
    df = df.dropna(axis=1, thresh=n_required_column)
    return df

# # Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [23]:
df = df.sort_values('transactiondate').drop_duplicates('parcelid',keep='last')

In [47]:
num_missing = df.isnull().sum()
pct_missing = df.isnull().sum()/df.shape[0] 

In [48]:
pd.DataFrame({'num_rows_missing': num_missing, 'pct_rows_missing': pct_missing}) 

,num_rows_missing,pct_rows_missing
id,0,0.0
parcelid,0,0.0
airconditioningtypeid,0,0.0
architecturalstyletypeid,0,0.0
basementsqft,0,0.0
...,...,...
buildingclassdesc,0,0.0
heatingorsystemdesc,0,0.0
propertylandusedesc,0,0.0
storydesc,0,0.0


In [39]:
overview(df)

--- Shape: (77380, 68)
--- Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77380 non-null  int64  
 1   parcelid                      77380 non-null  int64  
 2   airconditioningtypeid         77380 non-null  float64
 3   architecturalstyletypeid      77380 non-null  float64
 4   basementsqft                  77380 non-null  float64
 5   bathroomcnt                   77380 non-null  float64
 6   bedroomcnt                    77380 non-null  float64
 7   buildingclasstypeid           77380 non-null  float64
 8   buildingqualitytypeid         77380 non-null  float64
 9   calculatedbathnbr             77380 non-null  float64
 10  decktypeid                    77380 non-null  float64
 11  finishedfloor1squarefeet      77380 non-null  float64
 12  calculatedfinishedsquarefeet

In [8]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns=cols_to_remove)
    return df
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df
def data_prep(df, cols_to_remove=[], prop_required_column=.5, prop_required_row=.75):
    df = remove_columns(df, cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    return df


In [9]:
df=handle_missing_values(df)
df=data_prep(df)

In [40]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,0.0,0.0,0.0,3.5,4.0,0.0,0.0,3.5,...,6.059063e+13,0.025595,2017-01-01,0,0,0,0,Single Family Residential,0,0
1,1387261,17052889,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,...,6.111001e+13,0.055619,2017-01-01,0,0,0,0,Single Family Residential,0,0
2,11677,14186244,0.0,0.0,0.0,2.0,3.0,0.0,0.0,2.0,...,6.059022e+13,0.005383,2017-01-01,0,0,0,0,Single Family Residential,0,0
3,2288172,12177905,0.0,0.0,0.0,3.0,4.0,0.0,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,0,0,0,Central,Single Family Residential,0,0
4,1970746,10887214,1.0,0.0,0.0,3.0,3.0,0.0,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,0,0,Central,Condominium,0,0


In [17]:
df.nunique()

id                                         48649
parcelid                                   48649
bathroomcnt                                   26
bedroomcnt                                    16
buildingqualitytypeid                         12
                                           ...  
taxvaluedollarcnt_outliers_outliers         6026
assessmentyear_outliers_outliers               1
landtaxvaluedollarcnt_outliers_outliers     6071
taxamount_outliers_outliers                 7270
censustractandblock_outliers_outliers       2736
Length: 86, dtype: int64

In [12]:
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

def add_upper_outlier_columns(df, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    # outlier_cols = {col + '_outliers': get_upper_outliers(df[col], k)
    #                 for col in df.select_dtypes('number')}
    # return df.assign(**outlier_cols)

    for col in df.select_dtypes('number'):
        df[col + '_outliers'] = get_upper_outliers(df[col], k)

    return df



In [14]:
df = add_upper_outlier_columns(df,k=1.50)
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,numberofstories_outliers_outliers,fireplaceflag_outliers_outliers,structuretaxvaluedollarcnt_outliers_outliers,taxvaluedollarcnt_outliers_outliers,assessmentyear_outliers_outliers,landtaxvaluedollarcnt_outliers_outliers,taxamount_outliers_outliers,taxdelinquencyyear_outliers_outliers,censustractandblock_outliers_outliers,logerror_outliers_outliers
0,1727539,14297519,0.0,0.0,0.0,3.5,4.0,0.0,0.0,3.5,...,0.0,0.0,65460.125,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0
1,1387261,17052889,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,...,1.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,1.936079e+11,0.0
2,11677,14186244,0.0,0.0,0.0,2.0,3.0,0.0,0.0,2.0,...,1.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0
3,2288172,12177905,0.0,0.0,0.0,3.0,4.0,0.0,8.0,3.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0
4,1970746,10887214,1.0,0.0,0.0,3.0,3.0,0.0,8.0,3.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0


In [17]:
outlier_cols = [col for col in df if col.endswith('_outliers')]
for col in outlier_cols:
    print('~~~\n' + col)
    data = df[col][df[col] > 0]
    print(data.describe())

~~~
id_outliers
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: id_outliers, dtype: float64
~~~
parcelid_outliers
count    3.100000e+01
mean     1.435928e+08
std      2.021958e+07
min      6.141833e+07
25%      1.494141e+08
50%      1.494149e+08
75%      1.494167e+08
max      1.494672e+08
Name: parcelid_outliers, dtype: float64
~~~
airconditioningtypeid_outliers
count    1820.000000
mean        9.650549
std         2.345003
min         2.500000
25%        10.500000
50%        10.500000
75%        10.500000
max        10.500000
Name: airconditioningtypeid_outliers, dtype: float64
~~~
architecturalstyletypeid_outliers
count    206.000000
mean       7.388350
std        2.734542
min        2.000000
25%        7.000000
50%        7.000000
75%        7.000000
max       21.000000
Name: architecturalstyletypeid_outliers, dtype: float64
~~~
basementsqft_outliers
count      50.000000
mean      679.720000
std       689.703546
min      

# Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [19]:
def summery_df(df):
    print(df.nunique())
    print('----------------------------------------------------------------------------------------------------')
    print(df.info())
    print('----------------------------------------------------------------------------------------------------')
    print(df.describe())
    print('----------------------------------------------------------------------------------------------------')
    print(df.isnull().sum()/len(df)*100)
    print('----------------------------------------------------------------------------------------------------')
    print(df.isnull().sum(axis=0))
    print('----------------------------------------------------------------------------------------------------')
    print(df.shape)
    print('----------------------------------------------------------------------------------------------------')
    print(df.head())
    print('----------------------------------------------------------------------------------------------------')

In [20]:
summery_df(df)

id                                         48649
parcelid                                   48649
bathroomcnt                                   26
bedroomcnt                                    16
buildingqualitytypeid                         12
                                           ...  
taxvaluedollarcnt_outliers_outliers         6026
assessmentyear_outliers_outliers               1
landtaxvaluedollarcnt_outliers_outliers     6071
taxamount_outliers_outliers                 7270
censustractandblock_outliers_outliers       2736
Length: 86, dtype: int64
----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 48649 entries, 2 to 49999
Data columns (total 86 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              48649 non-null  int

# Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [26]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:

        q1, q3 = df[f'{col}'].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        return df[(df[f'{col}'] > lower_bound) & (df[f'{col}'] < upper_bound)]  

In [27]:
col_list=['id','parcelid','airconditioningtypeid','architecturalstyletypeid','basementsqft','bathroomcnt','bedroomcnt']
df = remove_outliers(df, 1.5, col_list)
df

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,None,None,Central,Condominium,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77375,2864704,10833991,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037132e+13,-0.002245,2017-09-20,Central,None,None,Central,Condominium,None,None
77376,673515,11000655,NaN,NaN,NaN,2.0,2.0,NaN,6.0,2.0,...,6.037101e+13,0.020615,2017-09-20,None,None,None,Central,Single Family Residential,None,None
77378,1843709,12773139,1.0,NaN,NaN,1.0,3.0,NaN,4.0,1.0,...,6.037434e+13,0.037129,2017-09-21,Central,None,None,Central,Single Family Residential,None,None
77377,2968375,17239384,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,...,6.111008e+13,0.013209,2017-09-21,None,None,None,None,Single Family Residential,None,None


# Mall Customers

Notebook

Acquire data from the customers table in the mall_customers database.

Summarize the data (include distributions and descriptive statistics).

Detect outliers using IQR.

Split data into train, validate, and test.

Encode categorical columns using a one hot encoder (pd.get_dummies).

Handles missing values.

Scaling

Encapsulate your work in a wrangle_mall.py python module.